# Autodesk Moldflow automation tutorial 

Autodesk Moldflow is a popular injection molding software. The Autodesk Moldflow Synergy GUI is able to record user macros for automation as Visual Basic Scripts (*.vbs). However, we may want to integrate this automation with a Python workflow. We could start subprocesses to call Moldflow with a *.vbs script, but it is a much more seamless experience to control Moldflow directly from Python. Luckily, that is possible and this Notebook illustrates how it is done. 

## 01 - Prerequisites 

First of all, we need a functioning Moldflow Insight installation with an active license purchased from [Autodesk](https://www.autodesk.com/products/moldflow/). Note that Moldflow Synergy is available for Microsoft Windows only.

In addition, we need to install the pywin32 package via pip or conda:
```
pip install pywin32
conda install pywin32
```

## 02 - Start Moldflow Synergy GUI from Python

Once the prequesites are met, we can fire up a Moldflow Synergy GUI instance with two simples lines of code: 

In [ ]:
# Import win32com.client module from pywin32 to use COM (Component Object Model)
import win32com.client

# Create Moldflow Synergy instance
Synergy = win32com.client.Dispatch("synergy.Synergy")

This should open a Moldflow Synergy Window on your machine. Pretty cool, isn´t it? In a next step, we learn how to interact with this Window and automate modifications to the model. 

## 03 - Convert VBS to Python Code 

The easiest way to continue from here is the creation of a regular VBS macro in Moldflow Synergy. 